<a href="https://colab.research.google.com/github/dutt2022/Coursera-Data-science-Projects/blob/main/TextRank_Eigen_Vector_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File ‘bbc_text_cls.csv’ already there; not retrieving.



In [ ]:
df = pd.read_csv('bbc_text_cls.csv')

In [ ]:
df

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business
...,...,...
2220,BT program to beat dialler scams\n\nBT is intr...,tech
2221,Spam e-mails tempt net shoppers\n\nComputer us...,tech
2222,Be careful how you code\n\nA new European dire...,tech
2223,US cyber security chief resigns\n\nThe man mak...,tech


In [ ]:
doc = df[df.labels == 'business']['text'].sample(random_state=42)

In [ ]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [ ]:
#print(wrap(doc.iloc[0][1]))

In [ ]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [ ]:
print(doc.iloc[0].split("\n", 1)[1])


UK retail sales fell in December, failing to meet expectations and making it by some counts the worst Christmas since 1981.

Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said. The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%. A number of retailers have already reported poor figures for December. Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth, according to the ONS.

The last time retailers endured a tougher Christmas was 23 years previously, when sales plunged 1.7%.

The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures. Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003. The November-December jump last year was roughl

In [ ]:
print(doc.iloc[0].split("\n", 1)[0])

Christmas sales worst since 1981


In [ ]:
doc = df[df.labels == 'business']['text'].sample(random_state=142)

In [ ]:
print(wrap(doc.iloc[0]))

French suitor holds LSE meeting

European stock market Euronext has
met with the London Stock Exchange (LSE) amid speculation that it may
be ready to launch a cash bid.

Euronext chief Jean-Francois Theodore
held talks with LSE boss Clara Furse the day after rival Deutsche
Boerse put forward its own bid case.  The German exchange said it had
held "constructive, professional and friendly" talks with the LSE. But
Euronext declined to comment after the talks ended on Friday.
Speculation is mounting that the Germans may raise their bid to
£1.5bn.  Deutsche Boerse previously offered £1.3bn, which was rejected
by the LSE, while Euronext is rumoured to have facilities in place to
fund a £1.4bn cash bid.  So far, however, neither have tabled a formal
bid.  But a deal with either bidder would create the biggest stock
market operator in Europe and the second biggest in the world after
the New York Stock Exchange.

There was speculation Euronext would use
Friday's meeting as an opportunity to tak

In [ ]:
sents = nltk.sent_tokenize(doc.iloc[0].split("\n", 1)[1])

In [ ]:
featurizer = TfidfVectorizer(
    stop_words=stopwords.words('english'),
    norm='l1')

In [ ]:
X = featurizer.fit_transform(sents)

In [ ]:
# compute similarity matrix
S = cosine_similarity(X)

In [ ]:
S.shape

(18, 18)

In [ ]:
len(sents)

18

In [ ]:
# normalize similarity matrix
S /= S.sum(axis=1, keepdims=True)

In [ ]:
S[0].sum()

1.0

In [ ]:
# uniform transition matrix
U = np.ones_like(S) / len(S)

In [ ]:
U[0].sum()

1.0

In [ ]:
# smoothed similarity matrix
factor = 0.15
S = (1 - factor) * S + factor * U

In [ ]:
S[0].sum()

1.0000000000000002

In [ ]:
# find the limiting / stationary distribution
eigenvals, eigenvecs = np.linalg.eig(S.T)

In [ ]:
eigenvals

array([1.        , 0.22594683, 0.72398087, 0.70713284, 0.64317234,
       0.31188529, 0.32236711, 0.34534314, 0.3525957 , 0.38432922,
       0.41005675, 0.45336931, 0.47507227, 0.48063928, 0.60266083,
       0.56721592, 0.55911263, 0.55717777])

In [ ]:
eigenvecs[:,0]

array([0.30183472, 0.25112939, 0.24048703, 0.20962727, 0.20972384,
       0.24480488, 0.19892056, 0.24143128, 0.26862991, 0.23959954,
       0.19707203, 0.24648145, 0.2580201 , 0.19526784, 0.21555003,
       0.21114237, 0.23613698, 0.24821829])

In [ ]:
eigenvecs[:,0].dot(S)

array([0.30183472, 0.25112939, 0.24048703, 0.20962727, 0.20972384,
       0.24480488, 0.19892056, 0.24143128, 0.26862991, 0.23959954,
       0.19707203, 0.24648145, 0.2580201 , 0.19526784, 0.21555003,
       0.21114237, 0.23613698, 0.24821829])

In [ ]:
eigenvecs[:,0] / eigenvecs[:,0].sum()

array([0.07162534, 0.05959297, 0.05706754, 0.04974452, 0.04976744,
       0.05809216, 0.04720382, 0.05729161, 0.06374584, 0.05685694,
       0.04676516, 0.05849001, 0.06122813, 0.04633703, 0.05114999,
       0.05010405, 0.05603527, 0.05890217])

In [ ]:
limiting_dist = np.ones(len(S)) / len(S)
threshold = 1e-8
delta = float('inf')
iters = 0
while delta > threshold:
  iters += 1

  # Markov transition
  p = limiting_dist.dot(S)

  # compute change in limiting distribution
  delta = np.abs(p - limiting_dist).sum()

  # update limiting distribution
  limiting_dist = p

print(iters)

43


In [ ]:
limiting_dist

array([0.07162534, 0.05959297, 0.05706754, 0.04974452, 0.04976744,
       0.05809216, 0.04720382, 0.05729161, 0.06374584, 0.05685694,
       0.04676517, 0.05849001, 0.06122813, 0.04633703, 0.05114999,
       0.05010405, 0.05603527, 0.05890216])

In [ ]:
limiting_dist.sum()

0.9999999999999998

In [ ]:
np.abs(eigenvecs[:,0] / eigenvecs[:,0].sum() - limiting_dist).sum()

2.1280428391667883e-08

In [ ]:
scores = limiting_dist

In [ ]:
sort_idx = np.argsort(-scores)

In [ ]:
# Many options for how to choose which sentences to include:

# 1) top N sentences
# 2) top N words
# 3) top X% sentences or top X% words
# 4) sentences with scores > average score
# 5) sentences with scores > factor * average score

# You also don't have to sort. May make more sense in order.

print("Generated summary:")
for i in sort_idx[:5]:
  print(wrap("%.2f: %s" % (scores[i], sents[i])))

Generated summary:
0.07: 
European stock market Euronext has met with the London Stock
Exchange (LSE) amid speculation that it may be ready to launch a cash
bid.
0.06: There was speculation Euronext would use Friday's meeting as an
opportunity to take advantage of growing disquiet over Deutsche
Boerse's own plans for dominance over the London market.
0.06: Meanwhile, LSE shareholders fear that Deutsche Boerse's control
over its Clearstream unit - the clearing house that processes
securities transactions - would create a monopoly situation.
0.06: Euronext chief Jean-Francois Theodore held talks with LSE boss
Clara Furse the day after rival Deutsche Boerse put forward its own
bid case.
0.06: Euronext, so far, has given little away as to what sweeteners it
will offer the LSE - Europe's biggest equity market - into a deal.


In [ ]:
print("Generated summary:")
for i in sort_idx[:10]:
  print(wrap("%.2f: %s" % (scores[i], sents[i])))

Generated summary:
0.07: 
European stock market Euronext has met with the London Stock
Exchange (LSE) amid speculation that it may be ready to launch a cash
bid.
0.06: There was speculation Euronext would use Friday's meeting as an
opportunity to take advantage of growing disquiet over Deutsche
Boerse's own plans for dominance over the London market.
0.06: Meanwhile, LSE shareholders fear that Deutsche Boerse's control
over its Clearstream unit - the clearing house that processes
securities transactions - would create a monopoly situation.
0.06: Euronext chief Jean-Francois Theodore held talks with LSE boss
Clara Furse the day after rival Deutsche Boerse put forward its own
bid case.
0.06: Euronext, so far, has given little away as to what sweeteners it
will offer the LSE - Europe's biggest equity market - into a deal.
0.06: German politicians are also said to be angry over the market
operator's promise to move its headquarters to London if a bid were
successful.
0.06: Deutsche Boerse 

In [ ]:
doc.iloc[0].split("\n")[0]

'French suitor holds LSE meeting'

In [ ]:
def summarize(text, factor = 0.15):
  # extract sentences
  sents = nltk.sent_tokenize(text)

  # perform tf-idf
  featurizer = TfidfVectorizer(
      stop_words=stopwords.words('english'),
      norm='l1')
  X = featurizer.fit_transform(sents)

  # compute similarity matrix
  S = cosine_similarity(X)

  # normalize similarity matrix
  S /= S.sum(axis=1, keepdims=True)

  # uniform transition matrix
  U = np.ones_like(S) / len(S)

  # smoothed similarity matrix
  S = (1 - factor) * S + factor * U

  # find the limiting / stationary distribution
  eigenvals, eigenvecs = np.linalg.eig(S.T)

  # compute scores
  scores = eigenvecs[:,0] / eigenvecs[:,0].sum()
  
  # sort the scores
  sort_idx = np.argsort(-scores)

  # print summary
  
  print("Generated summary:")
  for i in sort_idx[:15]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

In [ ]:
doc = df[df.labels == 'business']['text'].sample(random_state=123)
summarize(doc.iloc[0].split("\n", 1)[1])

Generated summary:
0.10: WPP in recent years has also bought firms such as Ogilvy &
Mather and Cordiant Communications.
0.10: 
UK advertising giant WPP has posted larger-than-expected annual
profits and predicted that it will outperform the market in 2005.
0.09: The long-term outlook looks "very favourable" because of media
and technology developments and the strength of the US economy, WPP
said.
0.09: During the year WPP bought US rival Grey Global, creating a
giant big enough to rival sector leader Omnicom.
0.09: Chief Executive Martin Sorrell on Friday told Reuters news
agency that WPP had submitted a proposal for United Business Media's
NOP World market research unit.
0.09: Pre-tax profits rose 15% from a year ago to reach £546m
($1.04bn), ahead of average analysts' forecasts of £532m.
0.09: Events such as the Olympics helped boost WPP's profits in 2004.
0.09: The company said the US Congressional elections and the FIFA
World Cup are likely to present advertising opportunities in t

In [ ]:
print(doc.iloc[0].split("\n", 1)[1])


UK advertising giant WPP has posted larger-than-expected annual profits and predicted that it will outperform the market in 2005.

Pre-tax profits rose 15% from a year ago to reach £546m ($1.04bn), ahead of average analysts' forecasts of £532m. Revenues were £4.3bn while the firm's operating margins were 14.1%, which it said could reach 14.8% by 2006. During the year WPP bought US rival Grey Global, creating a giant big enough to rival sector leader Omnicom.

Chief Executive Martin Sorrell on Friday told Reuters news agency that WPP had submitted a proposal for United Business Media's NOP World market research unit. Analysts say the unit sell could sell for up to £350m. WPP in recent years has also bought firms such as Ogilvy & Mather and Cordiant Communications. It also includes the firms Young & Rubicam and J Walter Thompson. Events such as the Olympics helped boost WPP's profits in 2004. The company said the US Congressional elections and the FIFA World Cup are likely to present ad

In [ ]:
print(doc.iloc[0])

Ad firm WPP's profits surge 15%

UK advertising giant WPP has posted larger-than-expected annual profits and predicted that it will outperform the market in 2005.

Pre-tax profits rose 15% from a year ago to reach £546m ($1.04bn), ahead of average analysts' forecasts of £532m. Revenues were £4.3bn while the firm's operating margins were 14.1%, which it said could reach 14.8% by 2006. During the year WPP bought US rival Grey Global, creating a giant big enough to rival sector leader Omnicom.

Chief Executive Martin Sorrell on Friday told Reuters news agency that WPP had submitted a proposal for United Business Media's NOP World market research unit. Analysts say the unit sell could sell for up to £350m. WPP in recent years has also bought firms such as Ogilvy & Mather and Cordiant Communications. It also includes the firms Young & Rubicam and J Walter Thompson. Events such as the Olympics helped boost WPP's profits in 2004. The company said the US Congressional elections and the FIFA Wo

In [ ]:
print(wrap(doc.iloc[0]))

Ad firm WPP's profits surge 15%

UK advertising giant WPP has posted
larger-than-expected annual profits and predicted that it will
outperform the market in 2005.

Pre-tax profits rose 15% from a year
ago to reach £546m ($1.04bn), ahead of average analysts' forecasts of
£532m.  Revenues were £4.3bn while the firm's operating margins were
14.1%, which it said could reach 14.8% by 2006. During the year WPP
bought US rival Grey Global, creating a giant big enough to rival
sector leader Omnicom.

Chief Executive Martin Sorrell on Friday told
Reuters news agency that WPP had submitted a proposal for United
Business Media's NOP World market research unit.  Analysts say the
unit sell could sell for up to £350m.  WPP in recent years has also
bought firms such as Ogilvy & Mather and Cordiant Communications.  It
also includes the firms Young & Rubicam and J Walter Thompson.  Events
such as the Olympics helped boost WPP's profits in 2004. The company
said the US Congressional elections and the FI

In [ ]:
doc = df[df.labels == 'entertainment']['text'].sample(random_state=123)
summarize(doc.iloc[0].split("\n", 1)[1])

Generated summary:
0.11: Goodrem, Green Day and the Black Eyed Peas took home two awards
each.
0.10: As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.
0.10: Other winners included Green Day, voted best group, and the
Black Eyed Peas.
0.10: The Black Eyed Peas won awards for best R 'n' B video and
sexiest video, both for Hey Mama.
0.10: Local singer and songwriter Missy Higgins took the title of
breakthrough artist of the year, with Australian Idol winner Guy
Sebastian taking the honours for best pop video.
0.10: The 21-year-old singer won the award for best female artist,
with Australian Idol runner-up Shannon Noll taking the title of best
male at the ceremony.
0.09: 
Pop singer Delta Goodrem has scooped one of the top individual
prizes at the first Australian MTV Music Awards.
0.08: The ceremony was held at the Luna Park fairground in Sydney
Harbour and was hosted by the Osbourne fam

In [ ]:
print(doc.iloc[0].split("\n", 1)[1])


Pop singer Delta Goodrem has scooped one of the top individual prizes at the first Australian MTV Music Awards.

The 21-year-old singer won the award for best female artist, with Australian Idol runner-up Shannon Noll taking the title of best male at the ceremony. Goodrem, known in both Britain and Australia for her role as Nina Tucker in TV soap Neighbours, also performed a duet with boyfriend Brian McFadden. Other winners included Green Day, voted best group, and the Black Eyed Peas. Goodrem, Green Day and the Black Eyed Peas took home two awards each. As well as best female, Goodrem also took home the Pepsi Viewers Choice Award, whilst Green Day bagged the prize for best rock video for American Idiot. The Black Eyed Peas won awards for best R 'n' B video and sexiest video, both for Hey Mama. Local singer and songwriter Missy Higgins took the title of breakthrough artist of the year, with Australian Idol winner Guy Sebastian taking the honours for best pop video. The VH1 First Music

In [ ]:
print(doc.iloc[0].split("\n", 1)[0])

Goodrem wins top female MTV prize


In [ ]:
#print(wrap(doc.iloc[0][0]))

In [ ]:
print(wrap(doc.iloc[0].split("\n", 1)[1]))


Pop singer Delta Goodrem has scooped one of the top individual prizes
at the first Australian MTV Music Awards.

The 21-year-old singer won
the award for best female artist, with Australian Idol runner-up
Shannon Noll taking the title of best male at the ceremony.  Goodrem,
known in both Britain and Australia for her role as Nina Tucker in TV
soap Neighbours, also performed a duet with boyfriend Brian McFadden.
Other winners included Green Day, voted best group, and the Black Eyed
Peas.  Goodrem, Green Day and the Black Eyed Peas took home two awards
each.  As well as best female, Goodrem also took home the Pepsi
Viewers Choice Award, whilst Green Day bagged the prize for best rock
video for American Idiot.  The Black Eyed Peas won awards for best R
'n' B video and sexiest video, both for Hey Mama.  Local singer and
songwriter Missy Higgins took the title of breakthrough artist of the
year, with Australian Idol winner Guy Sebastian taking the honours for
best pop video.  The VH1 First

In [ ]:
print(wrap(doc.iloc[0]))

Goodrem wins top female MTV prize

Pop singer Delta Goodrem has
scooped one of the top individual prizes at the first Australian MTV
Music Awards.

The 21-year-old singer won the award for best female
artist, with Australian Idol runner-up Shannon Noll taking the title
of best male at the ceremony.  Goodrem, known in both Britain and
Australia for her role as Nina Tucker in TV soap Neighbours, also
performed a duet with boyfriend Brian McFadden.  Other winners
included Green Day, voted best group, and the Black Eyed Peas.
Goodrem, Green Day and the Black Eyed Peas took home two awards each.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.  The Black Eyed Peas won awards for best R 'n' B
video and sexiest video, both for Hey Mama.  Local singer and
songwriter Missy Higgins took the title of breakthrough artist of the
year, with Australian Idol winner Guy Sebastian taking the honours f

In [ ]:
doc = df[df.labels == 'tech']['text'].sample(random_state=123)
summarize(doc.iloc[0].split("\n", 1)[1])

Generated summary:
0.07: The Commodore 64 sold more than any other single computer
system, even to this day.
0.07: In the late 1980s the firm was a great rival to Atari, which
produced its own range of home computers and is now a brand of video
games, formerly known as Infogrames.
0.07: The groundbreaking Commodore 64 computer elicits fond memories
for those who owned one back in the 1980s.
0.07: The Commodore 64, launched in 1982, was one of the first
affordable home PCs.
0.07: 
The once-famous Commodore computer brand could be resurrected
after being bought by a US-based digital music distributor.
0.07: Commodore International filed for bankruptcy in 1994 and was
sold to Dutch firm Tulip Computers.
0.07: In the chronology of home computing, Commodore was one of the
pioneers.
0.07: Commodore 64 enthusiasts have written emulators for Windows PC,
Apple Mac and even PDAs so that the original Commodore games can be
still run.
0.07: Tulip Computers sold several products under the Commodore

In [ ]:
print(wrap(doc.iloc[0]))

Commodore finds new lease of life

The once-famous Commodore computer
brand could be resurrected after being bought by a US-based digital
music distributor.

New owner Yeahronimo Media Ventures has not ruled
out the possibility of a new breed of Commodore computers.  It also
plans to develop a "worldwide entertainment concept" with the brand,
although details are not yet known.  The groundbreaking Commodore 64
computer elicits fond memories for those who owned one back in the
1980s.

In the chronology of home computing, Commodore was one of the
pioneers.

The Commodore 64, launched in 1982, was one of the first
affordable home PCs.  It was followed a few years later by the Amiga.
The Commodore 64 sold more than any other single computer system, even
to this day.  The brand languished somewhat in the 1990s.  Commodore
International filed for bankruptcy in 1994 and was sold to Dutch firm
Tulip Computers.  In the late 1980s the firm was a great rival to
Atari, which produced its own range

In [ ]:
print(wrap(doc.iloc[0].split("\n",1)[0]))

Commodore finds new lease of life


In [ ]:
print(wrap(doc.iloc[0].split("\n",1)[1]))


The once-famous Commodore computer brand could be resurrected after
being bought by a US-based digital music distributor.

New owner
Yeahronimo Media Ventures has not ruled out the possibility of a new
breed of Commodore computers.  It also plans to develop a "worldwide
entertainment concept" with the brand, although details are not yet
known.  The groundbreaking Commodore 64 computer elicits fond memories
for those who owned one back in the 1980s.

In the chronology of home
computing, Commodore was one of the pioneers.

The Commodore 64,
launched in 1982, was one of the first affordable home PCs.  It was
followed a few years later by the Amiga.  The Commodore 64 sold more
than any other single computer system, even to this day.  The brand
languished somewhat in the 1990s.  Commodore International filed for
bankruptcy in 1994 and was sold to Dutch firm Tulip Computers.  In the
late 1980s the firm was a great rival to Atari, which produced its own
range of home computers and is now a b

## Libraries for Text Summarization

In [ ]:
!pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [ ]:
summarizer = TextRankSummarizer()
parser = PlaintextParser.from_string(
    doc.iloc[0].split("\n", 1)[1],
    Tokenizer("english"))
summary = summarizer(parser.document, sentences_count=5)

In [ ]:
summary

(<Sentence: The groundbreaking Commodore 64 computer elicits fond memories for those who owned one back in the 1980s.>,
 <Sentence: In the chronology of home computing, Commodore was one of the pioneers.>,
 <Sentence: The Commodore 64, launched in 1982, was one of the first affordable home PCs.>,
 <Sentence: In the late 1980s the firm was a great rival to Atari, which produced its own range of home computers and is now a brand of video games, formerly known as Infogrames.>,
 <Sentence: The sale of Commodore is expected to be complete in three weeks in a deal worth over £17m.>)

In [ ]:
for s in summary:
  print(wrap(str(s)))

The groundbreaking Commodore 64 computer elicits fond memories for
those who owned one back in the 1980s.
In the chronology of home computing, Commodore was one of the
pioneers.
The Commodore 64, launched in 1982, was one of the first affordable
home PCs.
In the late 1980s the firm was a great rival to Atari, which produced
its own range of home computers and is now a brand of video games,
formerly known as Infogrames.
The sale of Commodore is expected to be complete in three weeks in a
deal worth over £17m.


In [ ]:
summarizer = LsaSummarizer()
summary = summarizer(parser.document, sentences_count=5)
for s in summary:
  print(wrap(str(s)))

The once-famous Commodore computer brand could be resurrected after
being bought by a US-based digital music distributor.
It also plans to develop a "worldwide entertainment concept" with the
brand, although details are not yet known.
The Commodore 64 sold more than any other single computer system, even
to this day.
Commodore International filed for bankruptcy in 1994 and was sold to
Dutch firm Tulip Computers.
Tulip Computers sold several products under the Commodore name,
including portable USB storage devices and digital music players.


In [ ]:
# https://radimrehurek.com/gensim_3.8.3/summarization/summariser.html
# https://arxiv.org/abs/1602.03606
# Parameters
# text (str) – Given text.
# ratio (float, optional) – Number between 0 and 1 that determines the
#     proportion of the number of sentences of the original text to be
#     chosen for the summary.
# word_count (int or None, optional) – Determines how many words will the
#     output contain. If both parameters are provided, the ratio will be
#     ignored.
# split (bool, optional) – If True, list of sentences will be returned.
#     Otherwise joined strings will bwe returned.
from gensim.summarization.summarizer import summarize
summary = summarize(doc.iloc[0].split("\n", 1)[1])
print(wrap(summary))

In the late 1980s the firm was a great rival to Atari, which produced
its own range of home computers and is now a brand of video games,
formerly known as Infogrames.
Tulip Computers sold several products
under the Commodore name, including portable USB storage devices and
digital music players.
